In [1]:
import config
import json
import pymysql
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import pearsonr, spearmanr
from collections import Counter
from scipy.spatial.distance import euclidean
import requests
import sys
import plotly.express as px
from pysolar.solar import*
import plotly.graph_objects as go

In [2]:
sid = "penguin_db"
con_saem = pymysql.connect(host = config.saem_db_m["host"],password = config.saem_db_m["password"],
                           user = config.saem_db_m["user"],db=sid,charset = "utf8")

In [3]:
# 알파 디비에 접속
sid_sc = "sc_datascience_db"
con_sc = pymysql.connect(host=config.sc_db["host"],user=config.sc_db['user'],
                         password=config.sc_db['password'],db=sid_sc,charset="utf8")

In [4]:
sido = pd.DataFrame({"pwp_loct_grp_cd": ['41', '42', '43', '44', '45', '46', '47', '48', '50'],
                     "지역구분": ["경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]})

In [5]:
cli_info = pd.read_sql("SELECT * FROM CLIMATE_LOCATION",con=con_sc)

In [7]:
cli_info2= cli_info.STN_ID[cli_info.STN_NM.isin(["서울","강릉","청주","대전","전주","광주","대구","부산","제주"])].values

In [8]:
cli_info3 =pd.DataFrame({"stn_id":cli_info2,"지역구분":["경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]})

In [9]:
cli_info4 =pd.merge(cli_info3,sido,on="지역구분")

In [10]:
may_cli_2020= pd.read_sql("SELECT * FROM HOURLY_CLIMATE_DATA WHERE DATE>='2020-05-01' and DATE<='2020-05-31'",con=con_sc)

In [11]:
may_cli_2019= pd.read_sql("SELECT * FROM HOURLY_CLIMATE_DATA WHERE DATE>='2019-05-01' and DATE<='2019-05-31'",con=con_sc)

In [12]:
april_cli_2020= pd.read_sql("SELECT * FROM HOURLY_CLIMATE_DATA WHERE DATE>='2020-04-01' and DATE<='2020-04-30'",con=con_sc)

In [13]:
april_cli_2019= pd.read_sql("SELECT * FROM HOURLY_CLIMATE_DATA WHERE DATE>='2019-04-01' and DATE<='2019-04-30'",con=con_sc)

In [14]:
may_cli_2020.RN[may_cli_2020.RN=="NA"]=np.nan
may_cli_2020.RN = np.float32(may_cli_2020.RN)

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
may_cli_2019.RN[may_cli_2019.RN=="NA"]=np.nan
may_cli_2019.RN = np.float32(may_cli_2019.RN)

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
con_saem.close()

In [105]:
may_cli2020_2= may_cli_2020[may_cli_2020.STN_ID.isin(cli_info4.stn_id.values)]

In [106]:
may_cli2019_2= may_cli_2019[may_cli_2019.STN_ID.isin(cli_info4.stn_id.values)]

In [107]:
may2020_rn= may_cli2020_2.pivot_table(index="STN_ID",values="RN",aggfunc=np.nanmean).reset_index()

In [108]:
may2019_rn =may_cli2019_2.pivot_table(index="STN_ID",values="RN",aggfunc=np.nanmean).reset_index()

In [109]:
may2020_rn.columns = ['stn_id', 'rn']
may2019_rn.columns = ['stn_id', 'rn']

In [110]:
may2020_rn2 = pd.merge(cli_info4,may2020_rn,on="stn_id")

In [111]:
may2019_rn2 = pd.merge(cli_info4,may2019_rn,on="stn_id")

In [112]:
may2020_dc = may_cli2020_2.pivot_table(index="DATE",values="DC10_TCA",columns="STN_ID",aggfunc=np.nanmean)

In [113]:
may2019_dc = may_cli2019_2.pivot_table(index="DATE",values="DC10_TCA",columns="STN_ID",aggfunc=np.nanmean)

In [170]:
np.where(may2020_dc.iloc[:,0]>=6)

(array([ 2,  4,  7,  8, 13, 14, 15, 17, 18, 19, 20, 21, 22, 24, 25],
       dtype=int64),)

In [193]:
cli_info4

,stn_id,지역구분,pwp_loct_grp_cd
0,108,경기도,41
1,159,강원도,42
2,143,충청북도,43
3,156,충청남도,44
4,133,전라북도,45
5,105,전라남도,46
6,131,경상북도,47
7,146,경상남도,48
8,184,제주도,50


Int64Index([105, 108, 131, 133, 143, 146, 156, 159, 184], dtype='int64', name='STN_ID')

In [114]:
may2020dc = []
for i in range(9):
    may2020dc+=[len(np.where(may2020_dc.iloc[:,i]>=6)[0])]
    

In [115]:
may2020dcc = pd.DataFrame({"stn_id":[105, 108, 131, 133, 143, 146, 156, 159, 184],"2020년":may2020dc})

In [116]:
may2019dc = []
for i in range(9):
    may2019dc+=[len(np.where(may2019_dc.iloc[:,i]>=6)[0])]

In [117]:
may2019dcc = pd.DataFrame({"stn_id":[105, 108, 131, 133, 143, 146, 156, 159, 184],"2019년":may2019dc})

In [118]:
cli_info5 = cli_info4.copy()

In [119]:
cli_info6 = pd.merge(cli_info5,may2020dcc, on="stn_id")

In [120]:
cli_info7 = pd.merge(cli_info6,may2019dcc, on ="stn_id")

In [121]:
cli_info7.to_csv("구름많은날.csv",encoding="euckr")

In [122]:
cli_info5

,stn_id,지역구분,pwp_loct_grp_cd
0,108,경기도,41
1,159,강원도,42
2,143,충청북도,43
3,156,충청남도,44
4,133,전라북도,45
5,105,전라남도,46
6,131,경상북도,47
7,146,경상남도,48
8,184,제주도,50


In [186]:
cli_info5

,지역구분,지역코드,2020년 구름
0,경기도,NaN,15
1,강원도,NaN,21
2,충청북도,NaN,19
3,충청남도,NaN,19
4,전라북도,NaN,17
5,전라남도,NaN,18
6,경상북도,NaN,19
7,경상남도,NaN,16
8,제주도,NaN,14


In [156]:
may2019_rn2

,stn_id,지역구분,pwp_loct_grp_cd,rn
0,108,경기도,41,0.787500
1,159,강원도,42,1.119118
2,143,충청북도,43,0.503922
3,156,충청남도,44,1.382456
4,133,전라북도,45,0.523529
5,105,전라남도,46,0.281250
6,131,경상북도,47,0.356140
7,146,경상남도,48,0.735484
8,184,제주도,50,0.541772


In [263]:
fig  = go.Figure()



fig.add_trace(go.Bar(
y=may2019_rn2.지역구분,
x=may2019_rn2.rn,
name = "2019년 5월",
marker_color="#62CEDB",
orientation='h'))

fig.add_trace(go.Bar(
y=may2020_rn2.지역구분,
x=may2020_rn2.rn,
name = "2020년 5월",
marker_color="#F11524",
orientation='h'))



fig.update_layout(barmode='group',yaxis={'categoryorder':'array','categoryarray':['충청북도','충청남도','제주도','전라북도',
                                                                                                     '전라남도','경상북도','경상남도','경기도',
                                                                                                     '강원도']})
fig.update_layout(title_text ="2019년 5월 강수량(mm/시) VS 2020년 5월 강수량(mm/시)")

In [264]:
fig.write_html("201920205월 평균 강수량_20200611_2.html")

In [ ]:
may2020_rn[""]

In [26]:
np.nanmedian(may_cli_2020.ICSR)

0.93

In [25]:
np.nanmean(may_cli_2019.ICSR)

1.4712785

In [27]:
np.nanmedian(may_cli_2019.ICSR)

1.3

In [12]:
yyy2020 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date >= '2019-01-01' and stnd_date <= '2019-12-31'",con=con_saem)

In [13]:
yyy2020[yo]

,id,pwp_id,stnd_date,mon_acml_pv_amt,lol_mon_acml_pv_amt,avg_pv_time,lol_avg_pv_time,rnk,pv_cnt,pwp_loct_grp_cd,rec_amt,smp_amt,reg_dtm
0,3309,410,2019-01-01,11778.40,11778.40,3.8379,3.8379,NaN,31,41,1.053634e+06,1.309454e+06,2020-04-01 00:54:51
1,3310,210,2019-01-01,11479.32,11479.32,3.7141,3.7141,NaN,31,41,1.027296e+06,1.275038e+06,2020-04-01 00:54:51
2,3311,447,2019-01-01,11292.92,11292.92,3.6538,3.6538,NaN,31,41,1.010662e+06,1.254371e+06,2020-04-01 00:54:51
3,3312,702,2019-01-01,69947.45,69947.45,3.6511,3.6511,NaN,31,41,5.383639e+06,7.769917e+06,2020-04-01 00:54:51
4,3313,817,2019-01-01,6007.66,6007.66,3.1404,3.1404,NaN,31,41,5.371839e+05,6.672818e+05,2020-04-01 00:54:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5752,10652191,1826,2019-09-01,9496.93,9496.93,3.1707,3.1707,NaN,30,44,6.531086e+05,7.555520e+05,2020-06-24 15:57:41
5753,10652192,1826,2019-11-01,8722.07,8722.07,2.9120,2.9120,NaN,30,44,4.763755e+05,7.134322e+05,2020-06-24 15:57:41
5754,10652193,1826,2019-12-01,7256.88,7256.88,2.3447,2.3447,NaN,31,44,4.237573e+05,6.102866e+05,2020-06-24 15:57:41
5755,10652195,1826,2019-07-01,7064.85,4243.99,2.2826,1.3712,NaN,31,44,5.379941e+05,5.651153e+05,2020-06-24 15:57:41


In [16]:
yyyyy= yyy2020[yyy2020.pwp_id<1200].pivot_table(index="stnd_date",values="avg_pv_time",aggfunc=np.mean).reset_index()

In [17]:
yyyyy

,stnd_date,avg_pv_time
0,2019-01-01,2.668687
1,2019-02-01,3.155054
2,2019-03-01,3.604231
3,2019-04-01,3.668147
4,2019-05-01,4.637449
5,2019-06-01,4.069754
6,2019-07-01,3.330057
7,2019-08-01,3.815159
8,2019-09-01,3.149654
9,2019-10-01,3.539309


In [20]:
yyyyy["date"] = [str(x.strftime("%m"))+"월" for x in yyyyy.stnd_date]

In [21]:
yyyyy.columns =['stnd_date', '발전시간', '월']

In [24]:
yyyyy["구분"] = np.repeat(np.nan,len(yyyyy))

In [147]:
colors = ["#62CEDB"] * 12
colors[4] = "#F11524" 

In [148]:
colors

['#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#F11524',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB',
 '#62CEDB']

In [ ]:
yyyyy.구분[yyyyy.월=="05월"] = "5월"

In [149]:
fig = go.Figure(data=[go.Bar(
x=yyyyy.월, y=yyyyy.발전시간,
marker_color=colors)])

In [150]:
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title="2019년 월별 평균 발전시간")

In [32]:
# fig= px.bar(yyyyy,x="월",y="발전시간",title="2019년 월별 평균 발전시간")
# fig.update_layout(barmode='group', xaxis_tickangle=-45)

In [151]:
fig.write_html("./2019년 발전시간 평균_20200611.html")

In [5]:
april_2019 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date = '2019-04-01'",con=con_saem)

In [6]:
may_2019 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date = '2019-05-01'",con=con_saem)

In [39]:
may_2019_counter= pd.DataFrame(Counter(may_2019.pwp_loct_grp_cd).items())

In [40]:
len(may_2019)

404

In [46]:
may_2019_counter.columns = ["pwp_loct_grp_cd","count"]

In [42]:
pd.merge(may_2019_counter,sido, on ="pwp_loct_grp_cd").to_csv("./2019may_count.csv",sep=",",encoding="euckr")

In [7]:
april_2020 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date = '2020-04-01'",con=con_saem)

In [44]:
april_2020_counter= pd.DataFrame(Counter(april_2020.pwp_loct_grp_cd).items())

In [50]:
april_2020_counter.columns = ["pwp_loct_grp_cd","count"]
pd.merge(april_2020_counter,sido, on ="pwp_loct_grp_cd").to_csv("./2020april_count.csv",sep=",",encoding="euckr")

In [51]:
len(april_2020)

661

In [9]:
may_2020 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date = '2020-05-01'",con=con_saem)

In [53]:
may_2020_counter= pd.DataFrame(Counter(may_2020.pwp_loct_grp_cd).items())

In [54]:
may_2020_counter.columns = ["pwp_loct_grp_cd","count"]

In [55]:
pd.merge(may_2020_counter,sido, on ="pwp_loct_grp_cd").to_csv("./2020may_count.csv",sep=",",encoding="euckr")

In [56]:
len(may_2020)

678

In [57]:
may_2018 = pd.read_sql("SELECT * FROM generation_log_monthly where stnd_date = '2018-05-01'",con=con_saem)

In [58]:
april_2019.pivot_table(index="pwp_loct_grp_cd",columns="stnd_date",values="avg_pv_time",aggfunc=np.median)

stnd_date,2019-04-01
pwp_loct_grp_cd,
41,4.16600
42,4.39520
43,4.21725
44,4.05250
45,4.17270
46,3.98080
47,4.29735
48,4.14180
50,4.40510


In [89]:
con_sc.close()

In [107]:
may2020_mean=may_2020[may_2020.avg_pv_time<=8].pivot_table(index="pwp_loct_grp_cd",columns="stnd_date",values="avg_pv_time",aggfunc=np.median).reset_index()

In [108]:
may2019_mean= may_2019[may_2019.avg_pv_time<=8].pivot_table(index="pwp_loct_grp_cd",columns="stnd_date",values="avg_pv_time",aggfunc=np.median).reset_index()

In [109]:
april2020_mean=april_2020[april_2020.avg_pv_time<=8].pivot_table(index="pwp_loct_grp_cd",columns="stnd_date",values="avg_pv_time",aggfunc=np.median).reset_index()

In [110]:
may2019_mean["year"] = np.repeat("2019년 5월",len(may2019_mean))
may2020_mean["year"] = np.repeat("2020년 5월",len(may2020_mean))
april2020_mean["year"] = np.repeat("2020년 4월",len(april2020_mean))

In [99]:
april2020_mean.columns

Index(['pwp_loct_grp_cd', 2020-04-01, 'year'], dtype='object', name='stnd_date')

In [111]:
may2019_mean.columns = ['pwp_loct_grp_cd', '발전시간', '날짜']
may2020_mean.columns = ['pwp_loct_grp_cd', '발전시간', '날짜']


In [112]:
april2020_mean.columns = ['pwp_loct_grp_cd','발전시간','날짜']

In [103]:
may2019_mean2 = pd.merge(may2019_mean,sido,on="pwp_loct_grp_cd")

In [104]:
may2020_mean2 = pd.merge(may2020_mean,sido,on="pwp_loct_grp_cd")

In [105]:
april2020_mean2 = pd.merge(april2020_mean,sido,on="pwp_loct_grp_cd")

In [199]:
zzzz = pd.merge(may2020_mean.append(may2019_mean),sido, on = "pwp_loct_grp_cd")

4.526017103777269

In [144]:
np.mean(may2020_mean.발전시간)

4.3225685265722875

In [70]:
may_cli_2019.DC10_TCA[may_cli_2019.DC10_TCA=="NA"]=np.nan
may_cli_2019.DC10_TCA = np.float32(may_cli_2019.DC10_TCA)

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
may_cli_2020.DC10_TCA[may_cli_2020.DC10_TCA=="NA"]=np.nan
may_cli_2020.DC10_TCA = np.float32(may_cli_2020.DC10_TCA)

C:\Users\hussu\Anaconda3\envs\env2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [72]:
may2020_dc= np.mean(may_cli_2020.pivot_table(index="STN_ID",columns="DATE",values="DC10_TCA",aggfunc=np.nanmean)).values

In [73]:
may2019_dc= np.mean(may_cli_2019.pivot_table(index="STN_ID",columns="DATE",values="DC10_TCA",aggfunc=np.nanmean)).values

In [74]:
may2020_dc[may2020_dc>=6]

array([8.157967 , 8.639653 , 7.320971 , 6.7586994, 9.472527 , 7.2934966,
       6.9478035, 9.673078 , 8.544413 , 6.560897 , 7.960623 , 6.4574184,
       6.142858 , 6.9750166, 8.585163 , 7.6707873, 6.166665 , 7.0796695],
      dtype=float32)

In [15]:
may2019_dc[may2019_dc>=6]

array([7.393771 , 7.1927643, 8.072344 , 8.612111 , 7.411588 , 6.813633 ,
       6.82778  , 9.754719 , 6.5480375], dtype=float32)

In [113]:
fig  = go.Figure()
fig.add_trace(go.Bar(
y=may2020_mean2.지역구분,
x=may2020_mean2.발전시간,
name = "2020년 5월",
marker_color='#F11524',
orientation = "h"))

fig.add_trace(go.Bar(
y=may2019_mean2.지역구분,
x=may2019_mean2.발전시간,
name = "2019년 5월",
marker_color='#62CEDB',
orientation='h'))

fig.update_layout(barmode='group',yaxis={'categoryorder':'array','categoryarray':['충청북도','충청남도','제주도','전라북도',
                                                                                                     '전라남도','경상북도','경상남도','경기도',
                                                                                                     '강원도']})

# fig.update_layout(barmode='group', xaxis_tickangle=-45,yaxis={'categoryorder':'array','categoryarray':['제주도','강원도','전라북도',
#                                                                                                      '경기도','경상북도','충청북도','충청남도',
#                                                                                                      '경상남도','전라남도']})
fig.update_layout(title_text ="2019년 5월 발전시간 VS 2020년 5월 발전시간")

In [114]:
fig  = go.Figure()
fig.add_trace(go.Bar(
y=may2020_mean2.지역구분,
x=may2020_mean2.발전시간,
name = "2020년 5월",
marker_color='#F11524',
orientation='h'))

fig.add_trace(go.Bar(
y=april2020_mean2.지역구분,
x=april2020_mean2.발전시간,
name = "2020년 4월",
marker_color='#62CEDB',
orientation = "h"))


fig.update_layout(barmode='group',yaxis={'categoryorder':'array','categoryarray':['충청북도','충청남도','제주도','전라북도',
                                                                                                     '전라남도','경상북도','경상남도','경기도',
                                                                                                     '강원도']})
fig.update_layout(title_text ="2020년 4월 발전시간 VS 2020년 5월 발전시간")

In [88]:
con_saem.close()

In [87]:
fig.write_html("./201905202005_mean_20200615_1.html")

In [120]:
px.bar(zzzz,x="지역구분",y="발전시간")

,pwp_loct_grp_cd,발전시간,날짜,지역구분
0,41,3.956541,2020년 5월,경기도
1,41,4.513275,2019년 5월,경기도
2,42,4.000062,2020년 5월,강원도
3,42,4.822606,2019년 5월,강원도
4,43,4.293508,2020년 5월,충청북도
5,43,4.293263,2019년 5월,충청북도
6,44,4.262045,2020년 5월,충청남도
7,44,4.219499,2019년 5월,충청남도
8,45,4.417117,2020년 5월,전라북도
9,45,4.641630,2019년 5월,전라북도


In [11]:
april_2020.pivot_table(index="pwp_loct_grp_cd",columns="stnd_date",values="avg_pv_time",aggfunc=np.mean)

stnd_date,2020-04-01
pwp_loct_grp_cd,
41,4.680268
42,4.800641
43,5.251380
44,5.337104
45,5.417604
46,5.416509
47,5.082843
48,5.172329
50,5.494089
